# Scaffold Generator

- Using a set of rules, generate a biochemical network allowing different species to associate with each other



## Simple ABC Model 

Assuming Specie A recruits both B and C together, however, A can be bound to B, or C, or both B and C at the same time. However, B and C do not associate unless taken together by A. 

Thus, AB + C -> ABC and AC + B -> ABC

All Forward Reactions: 
- A + B -> AB 
- A + C -> AC 
- AB + C -> ABC 
- AC + B -> ABC 

Representing this: 
A: {B, C, [B, C]} # binds to B, C and can further associate B and C at the same time

If A binds either to B or C, then the representation would be: 
A: {B, C} # binds to either B or C but never together 

## Rule-based scaffold binding 

Makes an assumption that binding must occur through a specific binding site for the scaffold 

This means a scaffold will have a list of potential binding sites e.g. 
- Scaffold A: 
    - Surfaces: [B1, B2] # B1 and B2 are the names of the binding site
    - Prominence: [B3, ] # the shape of 'Scaffold A' makes it capable of binding to a B3 binding site 

Then, we define all species in the system in such manner 
- Specie B: 
    - Surfaces: [, ] # No binding site offered by this protein 
    - Prominence: [B1, B3] # binds to either proteins exposing a B1 or B3 binding site